In [1]:

from mxnet import nd
from mxnet.gluon import nn

In [2]:
layer = nn.Dense(2)
layer

Dense(None -> 2, linear)

In [5]:

layer.initialize()
x = nd.random.uniform(-1,1,(3,4))
layer(x)
layer.weight.data()

/root/miniconda/lib/python3.6/site-packages/mxnet/gluon/parameter.py:689: UserWarning: Parameter 'dense0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/root/miniconda/lib/python3.6/site-packages/mxnet/gluon/parameter.py:689: UserWarning: Parameter 'dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)



[[ 0.00952624 -0.01501013  0.05958354  0.04705103]
 [-0.06005495 -0.02276454 -0.0578019   0.02074406]]
<NDArray 2x4 @cpu(0)>

In [6]:
net = nn.Sequential()
# Add a sequence of layers.
net.add(# Similar to Dense, it is not necessary to specify the input channels
        # by the argument `in_channels`, which will be  automatically inferred
        # in the first forward pass. Also, we apply a relu activation on the
        # output. In addition, we can use a tuple to specify a  non-square
        # kernel size, such as `kernel_size=(2,4)`
        nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
        # One can also use a tuple to specify non-symmetric pool and stride sizes
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        # The dense layer will automatically reshape the 4-D output of last
        # max pooling layer into the 2-D shape: (x.shape[0], x.size/x.shape[0])
        nn.Dense(120, activation="relu"),
        nn.Dense(84, activation="relu"),
        nn.Dense(10))
net

Sequential(
  (0): Conv2D(None -> 6, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (4): Dense(None -> 120, Activation(relu))
  (5): Dense(None -> 84, Activation(relu))
  (6): Dense(None -> 10, linear)
)

In [7]:

net.initialize()
# Input shape is (batch_size, color_channels, height, width)
x = nd.random.uniform(shape=(4,1,28,28))
y = net(x)
y.shape

(4, 10)

In [8]:
(net[0].weight.data().shape, net[5].bias.data().shape)

((6, 1, 5, 5), (84,))

In [9]:

class MixMLP(nn.Block):
    def __init__(self, **kwargs):
        # Run `nn.Block`'s init method
        super(MixMLP, self).__init__(**kwargs)
        self.blk = nn.Sequential()
        self.blk.add(nn.Dense(3, activation='relu'),
                     nn.Dense(4, activation='relu'))
        self.dense = nn.Dense(5)
    def forward(self, x):
        y = nd.relu(self.blk(x))
        print(y)
        return self.dense(y)

net = MixMLP()
net

MixMLP(
  (blk): Sequential(
    (0): Dense(None -> 3, Activation(relu))
    (1): Dense(None -> 4, Activation(relu))
  )
  (dense): Dense(None -> 5, linear)
)

In [10]:
net.initialize()
x = nd.random.uniform(shape=(2,2))
net(x)


[[0.0015199  0.         0.00076785 0.        ]
 [0.00133006 0.         0.00064367 0.        ]]
<NDArray 2x4 @cpu(0)>



[[ 3.5631478e-05 -1.5303296e-05  8.4307962e-05  5.4950986e-05
   2.9626513e-05]
 [ 3.1893280e-05 -1.3741388e-05  7.3266892e-05  4.8983617e-05
   2.7170252e-05]]
<NDArray 2x5 @cpu(0)>

In [11]:
net.blk[1].weight.data()


[[ 0.05553398  0.03005757  0.06864745]
 [-0.0042704  -0.03963442 -0.00615795]
 [ 0.02283095  0.05689853 -0.03313487]
 [-0.05078914 -0.06710886 -0.0379093 ]]
<NDArray 4x3 @cpu(0)>